In [1]:
!pip install telethon

In [2]:
import configparser
import json

from telethon.sync import TelegramClient
from telethon import connection

# для корректного переноса времени сообщений в json
from datetime import date, datetime

# классы для работы с каналами
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch

# класс для работы с сообщениями
from telethon.tl.functions.messages import GetHistoryRequest

In [3]:
# Считываем учетные данные
# config = configparser.ConfigParser()
# config.read("config.ini")

# Присваиваем значения внутренним переменным
api_id = 28346692
api_hash = 'df748fc21f36338ad6aeff4adb3c7f96'
username = '79393419148'

In [4]:
client = TelegramClient(username, api_id, api_hash)

client.start()
# proxy = (proxy_server, proxy_port, proxy_key)

# client = TelegramClient(username, api_id, api_hash,
#     connection=connection.ConnectionTcpMTProxyRandomizedIntermediate,
#     proxy=proxy)

<coroutine object AuthMethods._start at 0x7ec17dc3a2d0>

In [6]:
!pip install nest-asyncio

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
async def dump_all_messages(channel):
	"""Записывает json-файл с информацией о всех сообщениях канала/чата"""
	offset_msg = 0    # номер записи, с которой начинается считывание
	limit_msg = 100   # максимальное число записей, передаваемых за один раз

	all_messages = []   # список всех сообщений
	total_messages = 0
	total_count_limit = 0  # поменяйте это значение, если вам нужны не все сообщения

	class DateTimeEncoder(json.JSONEncoder):
		'''Класс для сериализации записи дат в JSON'''
		def default(self, o):
			if isinstance(o, datetime):
				return o.isoformat()
			if isinstance(o, bytes):
				return list(o)
			return json.JSONEncoder.default(self, o)

	while True:
		history = await client(GetHistoryRequest(
			peer=channel,
			offset_id=offset_msg,
			offset_date=None, add_offset=0,
			limit=limit_msg, max_id=0, min_id=0,
			hash=0))
		if not history.messages:
			break
		messages = history.messages

		for message in messages:
			all_messages.append(message.to_dict())

		offset_msg = messages[len(messages) - 1].id
		total_messages = len(all_messages)
		if total_count_limit != 0 and total_messages >= total_count_limit:
			break

	with open('channel_messages.json', 'w', encoding='utf8') as outfile:
		 json.dump(all_messages, outfile, ensure_ascii=False, cls=DateTimeEncoder)


In [8]:
async def main():
	url = input("Введите ссылку на канал или чат: ")
	channel = await client.get_entity(url)
	await dump_all_messages(channel)

In [9]:
async with client:
	client.loop.run_until_complete(main())

Введите ссылку на канал или чат: https://t.me/orethana
